In [ ]:
!pip install yfinance

In [ ]:
import yfinance as yf
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.preprocessing import MinMaxScaler 
import pickle 
from tqdm.notebook import tnrange

In [ ]:
stock = yf.download("GOOGL" , start = "2018-01-01" , interval = '1d')

In [ ]:
stock.shape

In [ ]:
stock.head(3)

In [ ]:
# Sort the stock points based on indexes just for confirmation 
stock.sort_index(inplace = True)

In [ ]:
# Remove any duplicate index 
stock = stock.loc[~stock.index.duplicated(keep='first')]

In [ ]:
stock.tail(3)

In [ ]:
# Check for missing values 
stock.isnull().sum()

In [ ]:
# Get the statistics of the stock
stock.describe()

In [ ]:
import plotly.graph_objects as go

# Check the trend in Closing Values 
plot = go.Figure()

plot.add_trace(go.Scatter(x = stock.index , y = stock['Close'] , mode = 'lines'))
plot.update_layout(height = 500 , width = 900, 
                  xaxis_title='Date' , yaxis_title='Close')
plot.show()

In [ ]:
# Check the trend in Volume Traded
plot = go.Figure()

plot.add_trace(go.Scatter(x = stock.index , y = stock['Volume'] , mode = 'lines'))
plot.update_layout(height = 500 , width = 900, 
                  xaxis_title='Date' , yaxis_title='Volume')
plot.show()

In [ ]:
stock['EMA10'] = stock['Close'].ewm(span=10, adjust=False).mean()
stock['EMA50'] = stock['Close'].ewm(span=50, adjust=False).mean()
stock['EMA100'] = stock['Close'].ewm(span=100, adjust=False).mean()
stock = stock[['Close' , 'Volume','EMA10']]
stock.head(3)

In [ ]:
# Confirm the Testing Set length 
test_length = stock[(stock.index >= '2020-09-01')].shape[0]


In [ ]:
def FeaturesAndTargets(stock, feature_length):
    X = []
    Y = []

    for i in tnrange(len(stock) - feature_length): 
        X.append(stock.iloc[i : i + feature_length,:].values)
        Y.append(stock["Close"].values[i+feature_length])

    X = np.array(X)
    Y = np.array(Y)

    return X , Y

In [ ]:
X , Y = FeaturesAndTargets(stock , 32)

In [ ]:
# Check the shapes
X.shape , Y.shape

In [ ]:
Xtrain , Xtest , Ytrain , Ytest = X[:-test_length] , X[-test_length:] , Y[:-test_length] , Y[-test_length:]

In [ ]:
Xtrain.shape , Ytrain.shape

In [ ]:
Xtest.shape , Ytest.shape

In [ ]:
class MultiDimensionScaler():
    def __init__(self):
        self.scalers = []

    def fit_transform(self , X):
        total_dims = X.shape[2]
        for i in range(total_dims):
            Scaler = MinMaxScaler()
            X[:, :, i] = Scaler.fit_transform(X[:, :, i])
            self.scalers.append(Scaler)
        return X

    def transform(self , X):
        for i in range(X.shape[2]):
            X[:, :, i] = self.scalers[i].transform(X[:,:,i])
        return X 

In [ ]:
Feature_Scaler = MultiDimensionScaler()
Xtrain = Feature_Scaler.fit_transform(Xtrain)
Xtest = Feature_Scaler.transform(Xtest)

In [ ]:
Target_Scaler = MinMaxScaler()
Ytrain = Target_Scaler.fit_transform(Ytrain.reshape(-1,1))
Ytest = Target_Scaler.transform(Ytest.reshape(-1,1))

In [ ]:
def save_object(obj , name : str):
    pickle_out = open(f"{name}.pck","wb")
    pickle.dump(obj, pickle_out)
    pickle_out.close()

def load_object(name : str):
    pickle_in = open(f"{name}.pck","rb")
    stock = pickle.load(pickle_in)
    return stock

In [ ]:
# Save your objects for future purposes 
save_object(Feature_Scaler , "Feature_Scaler")
save_object(Target_Scaler , "Target_Scaler")

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint , ReduceLROnPlateau

save_best = ModelCheckpoint("best_weights.h5", monitor='val_loss', save_best_only=True, save_weights_only=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.25,patience=4, min_lr=0.00001,verbose = 1)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense , Dropout , LSTM , Bidirectional

model = Sequential()

model.add(Bidirectional(LSTM(512 ,return_sequences=True , recurrent_dropout=0.1, input_shape=(32, 2))))
model.add(LSTM(512 ,recurrent_dropout=0.1))
model.add(Dropout(0.2))
model.add(Dense(256 , activation='elu'))
model.add(Dropout(0.2))
model.add(Dense(64 , activation='elu'))
model.add(Dense(32 , activation='elu'))
model.add(Dropout(0.2))
model.add(Dense(1 , activation='linear'))

In [ ]:
optimizer = tf.keras.optimizers.SGD(learning_rate = 0.002)
model.compile(loss='mse', optimizer=optimizer)

In [ ]:
history = model.fit(Xtrain, Ytrain,
            epochs=15,
            batch_size = 1,
            verbose=1,
            shuffle=False ,
            validation_stock=(Xtest , Ytest),
            callbacks=[reduce_lr , save_best])

In [ ]:
model.summary()

In [ ]:
model.load_weights("best_weights.h5")

Visualize prediction on Test Set

In [ ]:
Forecast = model.predict(Xtest)

In [ ]:
Forecast = Target_Scaler.inverse_transform(Forecast)
Actual = Target_Scaler.inverse_transform(Ytest)

In [ ]:
Forecast = np.squeeze(Forecast , axis = 1)
Actual = np.squeeze(Actual , axis = 1)

In [ ]:
test_stockframe_dict = {'Actual' : list(Actual) , 'Predicted' : list(Forecast)}
test_df = pd.stockFrame.from_dict(test_stockframe_dict)

test_df.index = stock.index[-test_length:]

In [ ]:
test_df.head()

In [ ]:
plot = go.Figure()

plot.add_trace(go.Scatter(x = test_df.index , y = Actual , mode = 'lines' , name='Actual'))
plot.add_trace(go.Scatter(x = test_df.index , y = Forecast , mode = 'lines' , name='Predicted'))
plot.show()

In [ ]:
Total_features = np.concatenate((Xtrain , Xtest) , axis = 0)

In [ ]:
Total_Targets = np.concatenate((Ytrain , Ytest) , axis = 0)

In [ ]:
Forecast = model.predict(Total_features)

In [ ]:
Forecast = Target_Scaler.inverse_transform(Forecast)
Actual = Target_Scaler.inverse_transform(Total_Targets)

In [ ]:
Forecast = np.squeeze(Forecast , axis = 1)
Actual = np.squeeze(Actual , axis = 1)

In [ ]:
plot = go.Figure()

plot.add_trace(go.Scatter(x = stock.index , y = Actual , mode = 'lines' , name='Actual'))
plot.add_trace(go.Scatter(x = stock.index , y = Forecast , mode = 'lines' , name='Predicted'))
plot.show()